In [ ]:
from scipy.io import loadmat
import pandas as pd
import os 
# print(len(data['stimuli']))
# print(len(data['triggers'][0]))
nb_subjects = 35
path_to_lefebvre_exp2 = '/Users/thomasnegrello/Desktop/autoneuro/datasets/Lefebvre_2017/BehavioralData/data_exp2/'
path_to_processed_lefebvre_exp2 = '/Users/thomasnegrello/Desktop/autoneuro/datasets/datasets_processed/dataset_lefebvre_exp2/'

In [ ]:
for id in range(1, nb_subjects+1):
    data = loadmat(path_to_lefebvre_exp2 + 'exp2_'+ str(id) + '.mat')

    # In Behavioral_Varialbles.m, the third, fifth and eigth columns are extracted and correspond to variables of interest.
    observations = data['data'][:,2]
    # In Behavioral_Varialbles.m, actions is rescaled 
    actions = data['data'][:,4]/2 +0.5
    rewards = data['data'][:,7]

    # Create new DataFrame using the generated columns
    fdf = pd.DataFrame({
        's': observations,
        'a': actions,
        'r': rewards,
        })
    
    # One hot encoding
    state2_map = {1: [1,0,0,0], 2: [0,1,0,0], 3: [0,0,1,0], 4: [0,0,0,1]}
    fdf['s'] = fdf['s'].map(state2_map)

    # Shifting and filling NaN for action and state column
    action_shifted = fdf['a'].shift(-1)
    action_shifted.iloc[-1] = -1
    observations_shifted = fdf['s'].shift(-1)
    observations_shifted.iloc[-1] = -1
    
    fdf['s_next']= observations_shifted
    fdf['a_next']= action_shifted
    
    subject_id = 'Sub' + str(id)
    
    # Create the directory for the subject
    subject_folder = os.path.join(path_to_processed_lefebvre_exp2, subject_id)
    os.mkdir(subject_folder)
    
    # Save the required columns to their respective files
    column_to_filename = {
        'r': 'rewards.csv',
        's': 'observations.csv',
        's_next': 'next_observations.csv',
        'a': 'actions.csv',
        'a_next': 'next_actions.csv'
    }

    for col, filename in column_to_filename.items():
        file_path = os.path.join(subject_folder, filename)
        fdf[col].to_csv(file_path, header=False, index=False)